In [24]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np


In [ ]:
%shell
%cd /content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
!git config --global user.email "007rohitjj04@gmail.com"
!git config --global user.name "Rohith04MVK"
!git add -A
!git commit -m "Still testing google colab"
!git push

/content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
[main d23a361] Still testing google colab
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite src/main.ipynb (63%)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 772 bytes | 772.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
